In [4]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("TCS_2023.csv",
                 parse_dates=["Date"],
                 index_col="Date")

In [5]:
def volatility_momentum(df):
  "Volatility Momentum Indicator:Bollinger Bands Relative Strength Index(RSI) "
  period=20
  df=df.copy()
  df['MA']=df['close'].rolling(window=period).mean()
  df['std']=df['close'].rolling(window=period).std()
  df['upper_bb']=df['MA']+(df['std']*2)
  df['lower_bb']=df['MA']-(df['std']*2)
  df['bb_width']=df['upper_bb']-df['lower_bb']
  df['bb_width']=df['bb_width'].replace(0,np.nan)
  df['dist_lower']=df['close']-df['lower_bb']
  df['dist_upper']=df['close']-df['upper_bb']

  rsi_input=(df['dist_lower']/df['bb_width'])*100
  rsi_input=rsi_input.fillna(50)
  delta=rsi_input.diff()
  gain=(delta.where(delta>0,0)).rolling(window=14).mean()
  loss=(-1*delta.where(delta<0,0)).rolling(window=14).mean()
  rs=gain/loss
  rsi=100-(100/(1+rs))
  return rsi


In [6]:
def price_trend_acc(df):
  "Price Trend And Acclerator"
  exp1=df['close'].ewm(span=12).mean()
  exp2=df['close'].ewm(span=26).mean()
  macd=exp1-exp2
  return macd

In [20]:
def stoch_osc(df):
  "Stochastic Oscillator"
  period=14
  low_min=df['low'].rolling(window=period).min()
  high_max=df['high'].rolling(window=period).max()
  df=df.copy()
  df['tr1']=df['high']-df['low']
  df['tr2']=abs(df['high']-df['close'].shift())
  df['tr3']=abs(df['low']-df['close'].shift())
  df['true_range']=df[['tr1','tr2','tr3']].max(axis=1)
  df['atr']=df['true_range'].rolling(window=period).mean()
  df['atr']=df['atr'].replace(0,np.nan)
  stoch_k=100*(df['close']-low_min)/(high_max-low_min)
  stoch_k=stoch_k.fillna(50)
  stoch_atr=stoch_k*(df['close']/df['atr'])
  return stoch_atr

In [8]:
def generate_signals(indicator,df):
  "Generate buy/sell signals from indicators"
  signals=pd.Series(0,index=df.index)
  signals[indicator<20]=1 #buy
  signals[indicator>80]=-1 #sell
  return signals

In [17]:
# Volatility Momentum
df=df.copy()
df = df.rename(columns={'Current Price': 'close', 'High': 'high', 'Low': 'low'})
vm = volatility_momentum(df)
vm_signals = generate_signals(vm,df)
print("Volatility Momentum Buy/Sell signals:", vm_signals.value_counts())
print("Total Buy:", (vm_signals == 1).sum())
print("Total Sell:", (vm_signals == -1).sum())

Volatility Momentum Buy/Sell signals: 0    1
Name: count, dtype: int64
Total Buy: 0
Total Sell: 0


In [18]:
# Price Trend Accelerator
pta = price_trend_acc(df)
pta_signals = generate_signals(pta, df)
print("Price Trend Accelerator Buy/Sell signals:", pta_signals.value_counts())
print("Total Buy:", (pta_signals == 1).sum())
print("Total Sell:", (pta_signals == -1).sum())

Price Trend Accelerator Buy/Sell signals: 1    1
Name: count, dtype: int64
Total Buy: 1
Total Sell: 0


In [21]:
# Stochastic Oscillator
so = stoch_osc(df)
so_signals = generate_signals(so, df)
print("Stochastic Oscillator Buy/Sell signals:", so_signals.value_counts())
print("Total Buy:", (so_signals == 1).sum())
print("Total Sell:", (so_signals == -1).sum())

Stochastic Oscillator Buy/Sell signals: 0    1
Name: count, dtype: int64
Total Buy: 0
Total Sell: 0


In [22]:
signals=generate_signals(so_signals,df)
print(signals.value_counts)

<bound method IndexOpsMixin.value_counts of Date
2026-01-01    1
dtype: int64>
